In [34]:
from datetime import datetime, timedelta
import time
from collections import namedtuple
import pandas as pd
import requests
import matplotlib.pyplot as plt




API_KEY = ''
#BASE_URL = "http://api.wunderground.com/api/{}/history_{}/q/NE/Lincoln.json"
BASE_URL = "https://api.weather.com/v2/pws/history/daily?stationId=KTXMCKIN47&format=json&units=m&date={}&apiKey={}"


target_date = datetime(2018, 6, 1)
features = ["date", "tempAvg", "dewptAvg", "pressureTrend", "humidityHigh", "humidityLow", "tempHigh","tempLow", "dewptHigh", "dewptLow", "pressureMax", "pressureMin", "precipTotal"]
#features = ["date", "tempAvg", "dewptAvg", "pressureTrend", "tempHigh","tempLow", "dewptHigh", "dewptLow", "pressureMax", "pressureMin", "precipTotal"]
DailySummary = namedtuple("DailySummary", features)


def extract_weather_data(url, api_key, target_date, days):
    records = []
    for _ in range(days):
        request = BASE_URL.format(target_date.strftime('%Y%m%d'), API_KEY)
        response = requests.get(request)
        observations = response.json().get('observations')
        if response.status_code == 200:
            if not observations:
                time.sleep(4)
                target_date += timedelta(days=1)
                continue
            data1 = response.json()['observations'][0]['metric']
            data2 = response.json()['observations'][0]
            records.append(DailySummary(
                date=target_date.strftime('%Y%m%d'),
                tempAvg=data1['tempAvg'],
                dewptAvg=data1['dewptAvg'],
                pressureTrend=data1['pressureTrend'],
                humidityHigh=data2['humidityHigh'],
                humidityLow=data2['humidityLow'],
                tempHigh=data1['tempHigh'],
                tempLow=data1['tempLow'],
                dewptHigh=data1['dewptHigh'],
                dewptLow=data1['dewptLow'],
                pressureMax=data1['pressureMax'],
                pressureMin=data1['pressureMin'],
                precipTotal=data1['precipTotal']
                )) 
        time.sleep(4)
        target_date += timedelta(days=1)
    return records



In [40]:
wdf = pd.read_pickle("./weather_data.pkl")

In [41]:
display(wdf)

,tempAvg,dewptAvg,pressureTrend,humidityHigh,humidityLow,tempHigh,tempLow,dewptHigh,dewptLow,pressureMax,pressureMin,precipTotal
date,,,,,,,,,,,,
20180226,12.0,6.0,1.69,97.0,34.0,19.0,6.0,8.0,3.0,1019.30,1013.55,0.00
20180227,14.0,12.0,-9.14,97.0,68.0,19.0,9.0,18.0,6.0,1015.92,1005.76,0.00
20180228,19.0,17.0,-2.37,97.0,82.0,21.0,17.0,19.0,16.0,1006.43,1000.34,10.92
20180301,14.0,9.0,15.92,94.0,49.0,17.0,10.0,16.0,1.0,1019.30,1002.71,0.00
20180302,13.0,3.0,-1.69,78.0,33.0,21.0,5.0,7.0,0.0,1022.01,1016.93,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
20201117,13.0,2.0,0.03,85.0,24.0,23.0,3.0,5.0,0.0,1025.94,1021.44,0.00
20201118,15.0,4.0,-0.20,76.0,28.0,23.0,8.0,6.0,2.0,1023.13,1016.93,0.00
20201119,18.0,11.0,-0.09,88.0,42.0,24.0,13.0,15.0,2.0,1018.42,1013.14,0.00


In [43]:
tmp = wdf[['tempAvg','dewptAvg']].head(10)
tmp

,tempAvg,dewptAvg
date,,
20180226,12.0,6.0
20180227,14.0,12.0
20180228,19.0,17.0
20180301,14.0,9.0
20180302,13.0,3.0
20180303,16.0,6.0
20180304,18.0,12.0
20180305,18.0,4.0
20180306,12.0,-6.0


In [44]:
N = 1

feature = 'tempAvg'

rows = tmp.shape[0]

nth_prior_measurements = [None]*N + [tmp[feature][i-N] for i in range(N, rows)]

col_name = "{}_{}".format(feature, N)
tmp[col_name] = nth_prior_measurements
tmp

,tempAvg,dewptAvg,tempAvg_1
date,,,
20180226,12.0,6.0,NaN
20180227,14.0,12.0,12.0
20180228,19.0,17.0,14.0
20180301,14.0,9.0,19.0
20180302,13.0,3.0,14.0
20180303,16.0,6.0,13.0
20180304,18.0,12.0,16.0
20180305,18.0,4.0,18.0
20180306,12.0,-6.0,18.0


In [48]:
def derive_nth_day_feature(df, feature, N):
    rows = df.shape[0]
    nth_prior_meassurements = [None]*N + [df[feature][i-N] for i in range(N, rows)]
    col_name = "{}_{}".format(feature, N)
    df[col_name] = nth_prior_meassurements

In [49]:
for feature in features:
    if feature != 'date':
        for N in range(1, 4):
            derive_nth_day_feature(wdf, feature, N)

In [50]:
wdf.columns

Index(['tempAvg', 'dewptAvg', 'pressureTrend', 'humidityHigh', 'humidityLow',
       'tempHigh', 'tempLow', 'dewptHigh', 'dewptLow', 'pressureMax',
       'pressureMin', 'precipTotal', 'tempAvg_1', 'tempAvg_2', 'tempAvg_3',
       'dewptAvg_1', 'dewptAvg_2', 'dewptAvg_3', 'pressureTrend_1',
       'pressureTrend_2', 'pressureTrend_3', 'humidityHigh_1',
       'humidityHigh_2', 'humidityHigh_3', 'humidityLow_1', 'humidityLow_2',
       'humidityLow_3', 'tempHigh_1', 'tempHigh_2', 'tempHigh_3', 'tempLow_1',
       'tempLow_2', 'tempLow_3', 'dewptHigh_1', 'dewptHigh_2', 'dewptHigh_3',
       'dewptLow_1', 'dewptLow_2', 'dewptLow_3', 'pressureMax_1',
       'pressureMax_2', 'pressureMax_3', 'pressureMin_1', 'pressureMin_2',
       'pressureMin_3', 'precipTotal_1', 'precipTotal_2', 'precipTotal_3'],
      dtype='object')

In [52]:
# make list of original features without meantempm, mintempm, and maxtempm
to_remove = [feature 
             for feature in features 
             if feature not in ['tempAvg', 'tempHigh', 'tempLow']]

# make a list of columns to keep
to_keep = [col for col in wdf.columns if col not in to_remove]

# select only the columns in to_keep and assign to df
df = wdf[to_keep]
df.columns

Index(['tempAvg', 'tempHigh', 'tempLow', 'tempAvg_1', 'tempAvg_2', 'tempAvg_3',
       'dewptAvg_1', 'dewptAvg_2', 'dewptAvg_3', 'pressureTrend_1',
       'pressureTrend_2', 'pressureTrend_3', 'humidityHigh_1',
       'humidityHigh_2', 'humidityHigh_3', 'humidityLow_1', 'humidityLow_2',
       'humidityLow_3', 'tempHigh_1', 'tempHigh_2', 'tempHigh_3', 'tempLow_1',
       'tempLow_2', 'tempLow_3', 'dewptHigh_1', 'dewptHigh_2', 'dewptHigh_3',
       'dewptLow_1', 'dewptLow_2', 'dewptLow_3', 'pressureMax_1',
       'pressureMax_2', 'pressureMax_3', 'pressureMin_1', 'pressureMin_2',
       'pressureMin_3', 'precipTotal_1', 'precipTotal_2', 'precipTotal_3'],
      dtype='object')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 990 entries, 20180226 to 20201121
Data columns (total 39 columns):
tempAvg            720 non-null float64
tempHigh           720 non-null float64
tempLow            720 non-null float64
tempAvg_1          719 non-null float64
tempAvg_2          718 non-null float64
tempAvg_3          717 non-null float64
dewptAvg_1         719 non-null float64
dewptAvg_2         718 non-null float64
dewptAvg_3         717 non-null float64
pressureTrend_1    989 non-null float64
pressureTrend_2    988 non-null float64
pressureTrend_3    987 non-null float64
humidityHigh_1     719 non-null float64
humidityHigh_2     718 non-null float64
humidityHigh_3     717 non-null float64
humidityLow_1      719 non-null float64
humidityLow_2      718 non-null float64
humidityLow_3      717 non-null float64
tempHigh_1         719 non-null float64
tempHigh_2         718 non-null float64
tempHigh_3         717 non-null float64
tempLow_1          719 non-null float64
tempLow_

In [54]:
df = df.apply(pd.to_numeric, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 990 entries, 20180226 to 20201121
Data columns (total 39 columns):
tempAvg            720 non-null float64
tempHigh           720 non-null float64
tempLow            720 non-null float64
tempAvg_1          719 non-null float64
tempAvg_2          718 non-null float64
tempAvg_3          717 non-null float64
dewptAvg_1         719 non-null float64
dewptAvg_2         718 non-null float64
dewptAvg_3         717 non-null float64
pressureTrend_1    989 non-null float64
pressureTrend_2    988 non-null float64
pressureTrend_3    987 non-null float64
humidityHigh_1     719 non-null float64
humidityHigh_2     718 non-null float64
humidityHigh_3     717 non-null float64
humidityLow_1      719 non-null float64
humidityLow_2      718 non-null float64
humidityLow_3      717 non-null float64
tempHigh_1         719 non-null float64
tempHigh_2         718 non-null float64
tempHigh_3         717 non-null float64
tempLow_1          719 non-null float64
tempLow_

In [56]:
# Call describe on df and transpose it due to the large number of columns
spread = df.describe().T

# precalculate interquartile range for ease of use in next calculation
IQR = spread['75%'] - spread['25%']

# create an outliers column which is either 3 IQRs below the first quartile or
# 3 IQRs above the third quartile
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max'] > (spread['75%']+3*IQR))

# just display the features containing extreme outliers
spread.loc[spread.outliers,]

,count,mean,std,min,25%,50%,75%,max,outliers
pressureTrend_1,989.0,-0.000212,1.919865,-11.51,-0.14,-0.01,0.11,17.61,True
pressureTrend_2,988.0,-0.000324,1.920834,-11.51,-0.14,-0.01,0.11,17.61,True
pressureTrend_3,987.0,-0.000233,1.921806,-11.51,-0.14,-0.01,0.11,17.61,True
humidityHigh_1,719.0,87.450626,9.784161,35.00,81.00,90.00,96.00,99.00,True
humidityHigh_2,718.0,87.442897,9.788785,35.00,81.00,90.00,96.00,99.00,True
humidityHigh_3,717.0,87.442120,9.795596,35.00,81.00,90.00,96.00,99.00,True
precipTotal_1,989.0,2.046451,11.104700,0.00,0.00,0.00,0.00,212.34,True
precipTotal_2,988.0,2.048522,11.110133,0.00,0.00,0.00,0.00,212.34,True
precipTotal_3,987.0,2.050598,11.115574,0.00,0.00,0.00,0.00,212.34,True


In [57]:
df.to_pickle("./weather_data_cleaned.pkl")